In [12]:
import torch
import numpy as np
from transformers import BertTokenizer, BertModel, RobertaTokenizer, RobertaModel
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import f1_score
from tabulate import tabulate


categories = ['rec.autos', 'comp.graphics', 'sci.space']
newsgroup = fetch_20newsgroups(subset='all', categories=categories, shuffle=True, remove=('headers', 'footers', 'quotes'))

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [14]:
def train(x_train, x_test, y_train, y_test, clf):
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    return f1_score(y_test, y_pred, average='weighted')

In [23]:
bertMap = ["base-uncased", "large-uncased", "roberta"]
clfMap = ["RndmForset", "GBM", "ADA"]
table = [[" "]+clfMap]
for i in range(len(bertMap)):
    tokenizer = 0
    model = 0
    if i==0:
        model_name='bert-base-uncased'
        tokenizer = BertTokenizer.from_pretrained(model_name)
        model = BertModel.from_pretrained(model_name).to(device)
    if i==1:
        model_name='bert-base-cased'
        tokenizer = BertTokenizer.from_pretrained(model_name)
        model = BertModel.from_pretrained(model_name).to(device)
    if i==2:
        model_name='roberta-base'
        tokenizer = RobertaTokenizer.from_pretrained(model_name)
        model = RobertaModel.from_pretrained(model_name).to(device)
    max_length = 512  # Set the maximum sequence length
    tokenized_texts = []
    for text in newsgroup.data:
        tokens = tokenizer.tokenize(text)
    # Truncate tokens if they exceed max_length - 2 (to account for [CLS] and [SEP])
        tokens = tokens[:max_length - 2]  
        tokens = ['[CLS]'] + tokens + ['[SEP]']
        tokenized_texts.append(tokens)

    input_ids = []
    for tokens in tokenized_texts:
    # Convert tokens to ids
        input_id = tokenizer.convert_tokens_to_ids(tokens)
    # Pad input_id with 0s if its length is less than max_length
        input_id += [0] * (max_length - len(input_id))
        input_ids.append(input_id)
    input_ids = torch.tensor(input_ids).to(device)

    with torch.no_grad():
        outputs = model(input_ids)

    last_hidden_states = outputs.last_hidden_state
    embeddings = last_hidden_states[:, 0, :]

    if device == torch.device("cuda:0"):
        embeddings = embeddings.cpu()

    dataVec = embeddings.numpy()
    X_train, X_test, y_train, y_test = train_test_split(dataVec, newsgroup.target, test_size=0.33)
    smallArr=[bertMap[i]]
    for j in range(len(clfMap)):
        clf = 0
        if j==0:
            clf = RandomForestClassifier()
        if j==1:
            clf = GradientBoostingClassifier(n_estimators=125)
        if j==2:
            clf = clf = AdaBoostClassifier(n_estimators=200,algorithm='SAMME')
        f1 = train(X_train, X_test, y_train, y_test, clf)
        smallArr.append = f1
    table.append(smallArr)
print(tabulate(table))

OutOfMemoryError: CUDA out of memory. Tried to allocate 4.32 GiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Of the allocated memory 6.23 GiB is allocated by PyTorch, and 121.69 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)